# Video Detection with Tensorflow 1.x (GPU Enabled)


In [0]:
%tensorflow_version 1.x

# Enable GPU acceleration by navigating to 

> Edit > Notebook Settings > Choose "GPU" for hardware acceleration.

Then run the cell below to make sure the GPU is setup. You should see 'Found GPU'

In [0]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()

if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')

print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


# **Mount** your google drive my using **'Mount drive'** from the files section.

Compile the model definition. Remember to change to the name of your folder.

In [0]:
%cd /content/drive/My Drive/RoadSignDetection/models/research/
!protoc object_detection/protos/*.proto --python_out=.

/content/drive/My Drive/RoadSignDetection/models/research


Set the environment

In [0]:
import os
os.environ['PYTHONPATH'] += ':/content/drive/My Drive/RoadSignDetection/models/research/:/content/drive/My Drive/RoadSignDetection/models/research/slim'

# By reaching this point the Environment is now setup. 

# Imports

In [0]:
import numpy as np
import os
import tensorflow as tf
import cv2
import time
from google.colab.patches import cv2_imshow
import glob

# from matplotlib import pyplot as plt
# from PIL import Image
# from IPython.display import display

In [0]:
# Import the object detection module.
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

In [0]:
%cd '/content/drive/My Drive/RoadSignDetection/models/research/object_detection'

/content/drive/My Drive/RoadSignDetection/models/research/object_detection


# Model Preparation

## Variables

Any model exported using the `export_inference_graph.py` tool can be loaded here simply by changing the path.

In [0]:
MODEL_NAME = 'inference_graph_1'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'

# Load a (frozen) Tensorflow model into memory.

In [0]:
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

    sess = tf.Session(graph=detection_graph)

## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `Speed limit (70km/h)`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

In [0]:
PATH_TO_LABELS = 'training/label_map.pbtxt'
NUM_CLASSES = 43

# Adding Labels and categories
# Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this
# corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping
# integers to appropriate string labels would be fine
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

# Replace VideoCapture path to your test video (Create a test_video folder in object_detection and put your test video in there) and change the output file path. Then exectute the cell to run object detection

In [0]:
# Loading video
cap = cv2.VideoCapture("test_videos/road_sign_test.avi")
width = int(cap.get(3))
height = int(cap.get(4))

# Defines the codec and creates VideoWriter object at specific output path
out = cv2.VideoWriter('test_videos/road_sign_Result.avi', cv2.VideoWriter_fourcc(*"XVID"), 24, (width, height))
#  (*"XVID") | (*"X264") | (*"MJPG")


with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:

        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
        # Each box represents a part of the image where a particular object was detected.
        detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')

        # Each score represent how level of confidence for each of the objects.
        # Score is shown on the result image, together with the class label.
        detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
        detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = detection_graph.get_tensor_by_name('num_detections:0')

        while videoCap.isOpened():
            # Capture frame-by-frame
            ret, frame = videoCap.read()

            if ret:
                  # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
                  image_np_expanded = np.expand_dims(frame, axis=0)

                  # Actual detection.
                  (boxes, scores, classes, num) = sess.run(
                      [detection_boxes, detection_scores, detection_classes, num_detections],
                      feed_dict={image_tensor: image_np_expanded})

                  # Visualization of the results of a detection.
                  vis_util.visualize_boxes_and_labels_on_image_array(
                      frame,
                      np.squeeze(boxes),
                      np.squeeze(classes).astype(np.int32),
                      np.squeeze(scores),
                      category_index,
                      use_normalized_coordinates=True, line_thickness=8)

                  # Saves for video
                  out.write(frame)

            else:
                break


videoCap.release()
out.release()
cv2.destroyAllWindows()